In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [129]:
df_homicidios = pd.read_excel('Datos de trabajo\homicidios_merge.xlsx')
df_poblacion = pd.read_excel('Datasets\poblacion.xlsx', sheet_name="DATOS")


## KPI 1
Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

In [83]:
def contar_victimas_por_semestre(dataframe_homicidios):
    
    # Aseguramos de que la columna 'FECHA' esté en formato datetime
    dataframe_homicidios['FECHA'] = pd.to_datetime(dataframe_homicidios['FECHA'])
    
    # Creamos una nueva columna llamada 'SEMESTRE' que contiene el número de semestre
    dataframe_homicidios['SEMESTRE'] = (dataframe_homicidios['FECHA'].dt.month - 1) // 6 + 1
    
    # Creamos una nueva columna llamada 'AÑO' que contiene el año
    dataframe_homicidios['AÑO'] = dataframe_homicidios['FECHA'].dt.year
    
    # Agrupamos por semestre y año, y sumamos las víctimas
    cantidad_homicidios = dataframe_homicidios.groupby(['AÑO', 'SEMESTRE'])['N_VICTIMAS'].sum().reset_index()
    
    return cantidad_homicidios

# Llamamos a la función con el DataFrame de homicidios
cantidad_homicidios = contar_victimas_por_semestre(df_homicidios)

# Muestra el nuevo DataFrame 'cantidad_homicidios'
print(cantidad_homicidios)

     AÑO  SEMESTRE  N_VICTIMAS
0   2016         1          61
1   2016         2          64
2   2017         1          63
3   2017         2          68
4   2018         1          71
5   2018         2          69
6   2019         1          53
7   2019         2          44
8   2020         1          27
9   2020         2          50
10  2021         1          48
11  2021         2          36


In [84]:
def contar_poblacion_por_anio(dataframe_poblacion):
    
    # Agrupamos por semestre y año, y sumamos las víctimas
    cantidad_poblacion = dataframe_poblacion.groupby(['AAAA'])['TOTAL'].sum().reset_index()
    
    return cantidad_poblacion

# Llamamos a la función con el DataFrame de 'df_poblacion'
cantidad_poblacion = contar_poblacion_por_anio(df_poblacion)

print(cantidad_poblacion)

   AAAA    TOTAL
0  2016  3059122
1  2017  3063728
2  2018  3068043
3  2019  3072029
4  2020  3075646
5  2021  3078836


In [85]:
# Fusionamos los DataFrames a través de la columna 'AÑO' y 'AAAA'
resultado = pd.merge(cantidad_homicidios, cantidad_poblacion, left_on='AÑO', right_on='AAAA')

# Eliminamos la columna redundante 'AAAA'
resultado.drop('AAAA', axis=1, inplace=True)

print(resultado)

     AÑO  SEMESTRE  N_VICTIMAS    TOTAL
0   2016         1          61  3059122
1   2016         2          64  3059122
2   2017         1          63  3063728
3   2017         2          68  3063728
4   2018         1          71  3068043
5   2018         2          69  3068043
6   2019         1          53  3072029
7   2019         2          44  3072029
8   2020         1          27  3075646
9   2020         2          50  3075646
10  2021         1          48  3078836
11  2021         2          36  3078836


In [86]:
# Cambiamos el nombre de la columna 'TOTAL'
resultado = resultado.rename(columns={'TOTAL': 'POBLACION'})
print(resultado)

     AÑO  SEMESTRE  N_VICTIMAS  POBLACION
0   2016         1          61    3059122
1   2016         2          64    3059122
2   2017         1          63    3063728
3   2017         2          68    3063728
4   2018         1          71    3068043
5   2018         2          69    3068043
6   2019         1          53    3072029
7   2019         2          44    3072029
8   2020         1          27    3075646
9   2020         2          50    3075646
10  2021         1          48    3078836
11  2021         2          36    3078836


In [87]:

# Agregamos una nueva columna llamada 'TASA_HOMICIDIOS' con el resultado de la fórmula
resultado['TASA_HOMICIDIOS'] = (resultado['N_VICTIMAS'] / resultado['POBLACION']) * 100000

print(resultado)


     AÑO  SEMESTRE  N_VICTIMAS  POBLACION  TASA_HOMICIDIOS
0   2016         1          61    3059122         1.994036
1   2016         2          64    3059122         2.092104
2   2017         1          63    3063728         2.056318
3   2017         2          68    3063728         2.219518
4   2018         1          71    3068043         2.314179
5   2018         2          69    3068043         2.248991
6   2019         1          53    3072029         1.725244
7   2019         2          44    3072029         1.432278
8   2020         1          27    3075646         0.877864
9   2020         2          50    3075646         1.625675
10  2021         1          48    3078836         1.559031
11  2021         2          36    3078836         1.169273


In [88]:
# El objetivo es reducir en un 10% la tasa de homicidios del semestre anterior
# Agregamos una nueva columna llamada 'TASA_OBJETIVO' con el 90% del resultado de 'TASA_HOMICIDIOS' de la fila anterior
resultado['TASA_OBJETIVO'] = resultado['TASA_HOMICIDIOS'].shift(1) * 0.9

print(resultado)


     AÑO  SEMESTRE  N_VICTIMAS  POBLACION  TASA_HOMICIDIOS  TASA_OBJETIVO
0   2016         1          61    3059122         1.994036            NaN
1   2016         2          64    3059122         2.092104       1.794633
2   2017         1          63    3063728         2.056318       1.882893
3   2017         2          68    3063728         2.219518       1.850686
4   2018         1          71    3068043         2.314179       1.997566
5   2018         2          69    3068043         2.248991       2.082761
6   2019         1          53    3072029         1.725244       2.024092
7   2019         2          44    3072029         1.432278       1.552720
8   2020         1          27    3075646         0.877864       1.289050
9   2020         2          50    3075646         1.625675       0.790078
10  2021         1          48    3078836         1.559031       1.463107
11  2021         2          36    3078836         1.169273       1.403128


In [89]:
# Agregamos una nueva columna llamada 'DIFERENCIA' con la diferencia entre 'TASA_HOMICIDIOS' y 'TASA_OBJETIVO'
resultado['DIFERENCIA'] = (resultado['TASA_HOMICIDIOS'] - resultado['TASA_OBJETIVO'])

print(resultado)

     AÑO  SEMESTRE  N_VICTIMAS  POBLACION  TASA_HOMICIDIOS  TASA_OBJETIVO  \
0   2016         1          61    3059122         1.994036            NaN   
1   2016         2          64    3059122         2.092104       1.794633   
2   2017         1          63    3063728         2.056318       1.882893   
3   2017         2          68    3063728         2.219518       1.850686   
4   2018         1          71    3068043         2.314179       1.997566   
5   2018         2          69    3068043         2.248991       2.082761   
6   2019         1          53    3072029         1.725244       2.024092   
7   2019         2          44    3072029         1.432278       1.552720   
8   2020         1          27    3075646         0.877864       1.289050   
9   2020         2          50    3075646         1.625675       0.790078   
10  2021         1          48    3078836         1.559031       1.463107   
11  2021         2          36    3078836         1.169273       1.403128   

In [91]:
# Si el valor en 'DIFERENCIA' es positivo, significa que 'TASA_HOMICIDIOS' <= 'TASA_OBJETIVO' y por lo tanto, la tasa de homicidios se redujo al menos un 10%, por lo tanto el objetivo está cumplido
# Agregamos una nueva columna llamada 'LOGRADO' que evalúa si se logró (1) o no se logró (0) el objetivo de reducción de la tasa de homicidios
resultado['LOGRADO'] = resultado['DIFERENCIA'].apply(lambda x: 1 if x <= 0 else 0)

resultado


,AÑO,SEMESTRE,N_VICTIMAS,POBLACION,TASA_HOMICIDIOS,TASA_OBJETIVO,DIFERENCIA,LOGRADO
0,2016,1,61,3059122,1.994036,NaN,NaN,0
1,2016,2,64,3059122,2.092104,1.794633,0.297471,0
2,2017,1,63,3063728,2.056318,1.882893,0.173425,0
3,2017,2,68,3063728,2.219518,1.850686,0.368832,0
4,2018,1,71,3068043,2.314179,1.997566,0.316612,0
5,2018,2,69,3068043,2.248991,2.082761,0.166230,0
6,2019,1,53,3072029,1.725244,2.024092,-0.298847,1
7,2019,2,44,3072029,1.432278,1.552720,-0.120442,1
8,2020,1,27,3075646,0.877864,1.289050,-0.411186,1
9,2020,2,50,3075646,1.625675,0.790078,0.835597,0


In [92]:
# Exportamos el DataFrame como archivo de Excel
resultado.to_excel('Datos de trabajo\\KPI_1.xlsx', index=False)

## KPI 2
Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año actual - Número de accidentes mortales con víctimas en moto en el año anterior) / (Número de accidentes mortales con víctimas en moto en el año anterior)


In [108]:
def contar_victimas_moto(dataframe_homicidios):
    # Agrupamos por vehículo y año, y sumamos las víctimas
    homicidios_vehiculo = dataframe_homicidios.groupby(['AÑO', 'VICTIMA'])['N_VICTIMAS'].sum().reset_index()
    
    return homicidios_vehiculo

# Llamamos a la función con el DataFrame de homicidios
homicidios_vehiculo = contar_victimas_moto(df_homicidios)

print(homicidios_vehiculo)

     AÑO      VICTIMA  N_VICTIMAS
0   2016         AUTO          13
1   2016    BICICLETA           4
2   2016       CARGAS           1
3   2016         MOTO          58
4   2016        MOVIL           1
5   2016    PASAJEROS           1
6   2016       PEATON          44
7   2017         AUTO          32
8   2017    BICICLETA           3
9   2017       CARGAS           2
10  2017         MOTO          44
11  2017  OBJETO FIJO           4
12  2017       PEATON          44
13  2018         AUTO          13
14  2018    BICICLETA           3
15  2018         MOTO          57
16  2018        MOVIL           4
17  2018    PASAJEROS           1
18  2018       PEATON          62
19  2019         AUTO           9
20  2019    BICICLETA           7
21  2019         MOTO          45
22  2019    PASAJEROS           1
23  2019       PEATON          35
24  2020         AUTO          13
25  2020    BICICLETA           5
26  2020         MOTO          27
27  2020       PEATON          32
28  2021      

In [120]:
# Filtramos las filas donde 'VICTIMA' es igual a 'MOTO'
homicidios_moto = homicidios_vehiculo[homicidios_vehiculo['VICTIMA'] == 'MOTO']

# Reindexamos
homicidios_moto = homicidios_moto.reset_index(drop=True)

print(homicidios_moto)

    AÑO VICTIMA  N_VICTIMAS
0  2016    MOTO          58
1  2017    MOTO          44
2  2018    MOTO          57
3  2019    MOTO          45
4  2020    MOTO          27
5  2021    MOTO          38


In [123]:
# El objetivo es reducir en un 7% la cantidad de victimas fatales en moto del año anterior
# Agregamos una nueva columna llamada 'OBJETIVO' donde determinamos que el objetivo es -0.07 (un 7% menos).
homicidios_moto['OBJETIVO'] = -0.07

# Calculamos la tasa de variación de victimas fatales en moto
homicidios_moto['TASA_VARIACIÓN'] = (homicidios_moto['N_VICTIMAS'] - homicidios_moto['N_VICTIMAS'].shift(1)) / homicidios_moto['N_VICTIMAS'].shift(1)

print(homicidios_moto)

    AÑO VICTIMA  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN
0  2016    MOTO          58     -0.07             NaN
1  2017    MOTO          44     -0.07       -0.241379
2  2018    MOTO          57     -0.07        0.295455
3  2019    MOTO          45     -0.07       -0.210526
4  2020    MOTO          27     -0.07       -0.400000
5  2021    MOTO          38     -0.07        0.407407


In [124]:
# Si el valor en 'TASA_VARIACIÓN' es menor o igual -0,07, significa que la cantidad de victimas en moto se redujo al menos un 7%, por lo tanto el objetivo está cumplido
# Agregamos una nueva columna llamada 'LOGRADO' que evalúa si se logró (1) o no se logró (0) el objetivo de reducción de la tasa de homicidios
homicidios_moto['LOGRADO'] = homicidios_moto['TASA_VARIACIÓN'].apply(lambda x: 1 if x <= -0.07 else 0)

print(homicidios_moto)

    AÑO VICTIMA  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN  LOGRADO
0  2016    MOTO          58     -0.07             NaN        0
1  2017    MOTO          44     -0.07       -0.241379        1
2  2018    MOTO          57     -0.07        0.295455        0
3  2019    MOTO          45     -0.07       -0.210526        1
4  2020    MOTO          27     -0.07       -0.400000        1
5  2021    MOTO          38     -0.07        0.407407        0


In [125]:
# Exportamos el DataFrame como archivo de Excel
homicidios_moto.to_excel('Datos de trabajo\\KPI_2.xlsx', index=False)

## KPI 3
Reducir en un 5% la cantidad de siniestros fatales ocacionados en avenidas en el último año, en CABA, respecto del año anterior.

Definimos a la cantidad de accidentes mortales de siniestros viales en avenidas como el número absoluto de accidentes fatales que ocurrieron en una avenida en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales en avenidas es: (Número de accidentes mortales en avenidas en el año actual - Número de accidentes mortales en avenidas en el año anterior) / (Número de accidentes mortales en avenidas en el año anterior)

## KPI 4
Reducir en un 5% la cantidad de siniestros fatales ocacionados en calles en el último año, en CABA, respecto del año anterior.

Definimos a la cantidad de accidentes mortales de siniestros viales en calles como el número absoluto de accidentes fatales que ocurrieron en una calle en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales en calles es: (Número de accidentes mortales en calles en el año actual - Número de accidentes mortales en calles en el año anterior) / (Número de accidentes mortales en calles en el año anterior)

In [140]:
def contar_victimas_tipo_calle(dataframe_homicidios):
    # Agrupamos por vehículo y año, y sumamos las víctimas
    homicidios_tipo_calle = dataframe_homicidios.groupby(['AAAA', 'TIPO_DE_CALLE'])['N_VICTIMAS'].sum().reset_index()
    
    return homicidios_tipo_calle

# Llamamos a la función con el DataFrame de homicidios
homicidios_tipo_calle = contar_victimas_tipo_calle(df_homicidios)

print(homicidios_tipo_calle)

    AAAA TIPO_DE_CALLE  N_VICTIMAS
0   2016     AUTOPISTA          13
1   2016       AVENIDA          89
2   2016         CALLE          15
3   2016      GRAL PAZ           8
4   2017     AUTOPISTA          10
5   2017       AVENIDA          89
6   2017         CALLE          24
7   2017      GRAL PAZ           8
8   2018     AUTOPISTA           7
9   2018       AVENIDA          98
10  2018         CALLE          28
11  2018      GRAL PAZ           7
12  2019     AUTOPISTA           6
13  2019       AVENIDA          63
14  2019         CALLE          25
15  2019      GRAL PAZ           3
16  2020     AUTOPISTA           4
17  2020       AVENIDA          56
18  2020         CALLE          12
19  2020      GRAL PAZ           5
20  2021     AUTOPISTA          13
21  2021       AVENIDA          55
22  2021         CALLE          11
23  2021      GRAL PAZ           5


In [142]:
# Filtramos las filas donde 'TIPO_DE_CALLE' es igual a 'AVENIDA' para el KPI 3 y donde es igual a 'CALLE' para el KPI 4
homicidios_avenida = homicidios_tipo_calle[homicidios_tipo_calle['TIPO_DE_CALLE'] == 'AVENIDA']
homicidios_calle = homicidios_tipo_calle[homicidios_tipo_calle['TIPO_DE_CALLE'] == 'CALLE']

# Reindexamos
homicidios_avenida = homicidios_avenida.reset_index(drop=True)
homicidios_calle = homicidios_calle.reset_index(drop=True)

print(homicidios_avenida)
print(homicidios_calle)

   AAAA TIPO_DE_CALLE  N_VICTIMAS
0  2016       AVENIDA          89
1  2017       AVENIDA          89
2  2018       AVENIDA          98
3  2019       AVENIDA          63
4  2020       AVENIDA          56
5  2021       AVENIDA          55
   AAAA TIPO_DE_CALLE  N_VICTIMAS
0  2016         CALLE          15
1  2017         CALLE          24
2  2018         CALLE          28
3  2019         CALLE          25
4  2020         CALLE          12
5  2021         CALLE          11


In [143]:
# El objetivo es reducir en un 5% la cantidad de victimas fatales en avenida y en calle respecto del año anterior
# Agregamos una nueva columna llamada 'OBJETIVO' donde determinamos que el objetivo es -0.05 (un 5% menos).
homicidios_avenida['OBJETIVO'] = -0.05
homicidios_calle['OBJETIVO'] = -0.05

# Calculamos la tasa de variación de victimas fatales en avenida
homicidios_avenida['TASA_VARIACIÓN'] = (homicidios_avenida['N_VICTIMAS'] - homicidios_avenida['N_VICTIMAS'].shift(1)) / homicidios_avenida['N_VICTIMAS'].shift(1)

# Calculamos la tasa de variación de victimas fatales en calle
homicidios_calle['TASA_VARIACIÓN'] = (homicidios_calle['N_VICTIMAS'] - homicidios_calle['N_VICTIMAS'].shift(1)) / homicidios_calle['N_VICTIMAS'].shift(1)


print(homicidios_avenida)
print(homicidios_calle)

   AAAA TIPO_DE_CALLE  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN
0  2016       AVENIDA          89     -0.05             NaN
1  2017       AVENIDA          89     -0.05        0.000000
2  2018       AVENIDA          98     -0.05        0.101124
3  2019       AVENIDA          63     -0.05       -0.357143
4  2020       AVENIDA          56     -0.05       -0.111111
5  2021       AVENIDA          55     -0.05       -0.017857
   AAAA TIPO_DE_CALLE  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN
0  2016         CALLE          15     -0.05             NaN
1  2017         CALLE          24     -0.05        0.600000
2  2018         CALLE          28     -0.05        0.166667
3  2019         CALLE          25     -0.05       -0.107143
4  2020         CALLE          12     -0.05       -0.520000
5  2021         CALLE          11     -0.05       -0.083333


In [145]:
# Si el valor en 'TASA_VARIACIÓN' es menor o igual -0,05, significa que la cantidad de victimas en el tipo de calle se redujo al menos un 5%, por lo tanto el objetivo está cumplido
# Agregamos una nueva columna llamada 'LOGRADO' que evalúa si se logró (1) o no se logró (0) el objetivo de reducción de la tasa de homicidios
homicidios_avenida['LOGRADO'] = homicidios_avenida['TASA_VARIACIÓN'].apply(lambda x: 1 if x <= -0.05 else 0)
homicidios_calle['LOGRADO'] = homicidios_calle['TASA_VARIACIÓN'].apply(lambda x: 1 if x <= -0.05 else 0)

print(homicidios_avenida)
print(homicidios_calle)

   AAAA TIPO_DE_CALLE  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN  LOGRADO
0  2016       AVENIDA          89     -0.05             NaN        0
1  2017       AVENIDA          89     -0.05        0.000000        0
2  2018       AVENIDA          98     -0.05        0.101124        0
3  2019       AVENIDA          63     -0.05       -0.357143        1
4  2020       AVENIDA          56     -0.05       -0.111111        1
5  2021       AVENIDA          55     -0.05       -0.017857        0
   AAAA TIPO_DE_CALLE  N_VICTIMAS  OBJETIVO  TASA_VARIACIÓN  LOGRADO
0  2016         CALLE          15     -0.05             NaN        0
1  2017         CALLE          24     -0.05        0.600000        0
2  2018         CALLE          28     -0.05        0.166667        0
3  2019         CALLE          25     -0.05       -0.107143        1
4  2020         CALLE          12     -0.05       -0.520000        1
5  2021         CALLE          11     -0.05       -0.083333        1


In [146]:
# Exportamos el DataFrame como archivo de Excel
homicidios_avenida.to_excel('Datos de trabajo\\KPI_3.xlsx', index=False)
homicidios_calle.to_excel('Datos de trabajo\\KPI_4.xlsx', index=False)